# query update

> query update functions

In [ ]:
#| default_exp query_update

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from emb_opt.imports import *
from emb_opt.utils import pack_dataset, whiten
from emb_opt.backends.hf import HFDatabase
from emb_opt.core import Score

/home/dmai/miniconda3/envs/emb_opt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Query Update

The role of `QueryUpdate` is to take a set of `query_vectors`, a scored `query_dataset` and generate a new set of `query_vectors`

In [ ]:
#| export

class QueryUpdate():
    'Query update base class'
    def __call__(self, 
                 query_vectors: np.ndarray, # query vectors
                 query_dataset: Dataset # scored dataset
                ) -> np.ndarray: # new query vectors
        return query_vectors

## Reinforcement Learning

`RLUpdate` uses basic reinforcement learning to update query vectors. For each query, we gather the `embedding` and `score` of each item returned. We `whiten` the scores so that above average scores have a positive value and below average scores have a negative value. Our gradient update tells the query vectory to move closer to positive value scores and further from negative value scores

In [ ]:
#| export

class RLUpdate(QueryUpdate):
    'Reinforcement Learning update'
    def __init__(self, 
                 lr: float # learning rate
                ):
        self.lr = lr
        
    def __call__(self, 
                 query_vectors: np.ndarray, # query vectors
                 query_dataset: Dataset # scored dataset
                ) -> np.ndarray: # new query vectors
        
        packed_dict = pack_dataset(query_dataset, 'query_idx', ['embedding', 'score'])
        grads = []
        
        for query_idx in range(query_vectors.shape[0]):
            embs = np.array(packed_dict[query_idx]['embedding'])
            scores = np.array(packed_dict[query_idx]['score'])

            advantages = whiten(scores)
            grad = (advantages[:,None] * (2*(query_vectors[query_idx][None] - embs))).mean(0)
            grads.append(grad)

        grads = np.array(grads)
        updated_query_vectors = query_vectors - self.lr*grads
        return updated_query_vectors

In [ ]:
def dummy_score(row):
    return np.linalg.norm(row['embedding'])

vectors = np.random.randn(128, 256)
vector_dataset = Dataset.from_list([{'embedding' : i} for i in vectors])
vector_dataset.add_faiss_index('embedding')

db = HFDatabase(vector_dataset, 'embedding', 10)
score = Score(dummy_score)
update_strategy = RLUpdate(0.5)


query_vectors = np.random.randn(3, 256)/10
query_dataset = db.query(query_vectors)
query_dataset = score(query_dataset)
updated_queries = update_strategy(query_vectors, query_dataset)

assert np.all(np.linalg.norm(updated_queries, axis=-1) > np.linalg.norm(query_vectors, axis=-1))

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1813.36it/s]


## Top K

`TopKUpdate` creates a new `query_vector` from the top `k` results returned. If `score_weighting=True`, a score-weighted average is used.

In [ ]:
#| export

class TopKUpdate(QueryUpdate):
    'Top K update'
    def __init__(self, 
                 k: int, # top k value
                 score_weighting: bool=True # if True, top k average is weighted by score
                ):
        self.k = k
        self.score_weighting = score_weighting
        
    def __call__(self, 
                 query_vectors: np.ndarray, # query vectors
                 query_dataset: Dataset # scored dataset
                ) -> np.ndarray: # new query vectors
        
        packed_dict = pack_dataset(query_dataset, 'query_idx', ['embedding', 'score'])
        new_queries = []
        
        for query_idx in range(query_vectors.shape[0]):
            embs = np.array(packed_dict[query_idx]['embedding'])
            scores = np.array(packed_dict[query_idx]['score'])

            topk_idxs = scores.argsort()[::-1][:self.k]
            topk_embs = embs[topk_idxs]
            topk_scores = scores[topk_idxs]

            if self.score_weighting:
                new_queries.append(np.average(topk_embs, 0, weights=topk_scores))
            else:
                new_queries.append(np.average(topk_embs, 0))

        query_vectors = np.array(new_queries)
        
        return query_vectors

In [ ]:
def dummy_score(row):
    return np.linalg.norm(row['embedding'])

vectors = np.random.randn(128, 256)
vector_dataset = Dataset.from_list([{'embedding' : i} for i in vectors])
vector_dataset.add_faiss_index('embedding')

db = HFDatabase(vector_dataset, 'embedding', 10)
score = Score(dummy_score)
update_strategy = TopKUpdate(3)


query_vectors = np.random.randn(3, 256)/10
query_dataset = db.query(query_vectors)
query_dataset = score(query_dataset)
updated_queries = update_strategy(query_vectors, query_dataset)

assert np.all(np.linalg.norm(updated_queries, axis=-1) > np.linalg.norm(query_vectors, axis=-1))

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1978.45it/s]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()